# ECOLANG Mesh Generation API
## Google Colab Backend for Real-time Mesh Rendering

This notebook creates a FastAPI server that:
- Reads NPZ parameters from Google Drive
- Generates 3D mesh frames using SMPL-X
- Exposes API endpoints via ngrok tunnel
- Serves mesh images to Streamlit Cloud frontend

**Important:** Keep this notebook running while using the ECOLANG app!

In [ ]:
# Cell 1: Install Dependencies
print("Installing dependencies...")
!pip install -q fastapi uvicorn pyngrok pillow trimesh pyrender smplx torch opencv-python nest-asyncio httpx
print("✓ Dependencies installed")

In [ ]:
# Cell 2: Mount Google Drive
from google.colab import drive
import os

drive.mount('/content/drive')

# Verify folder structure
BASE_PATH = "/content/drive/MyDrive/ecolang"
MODEL_PATH = os.path.join(BASE_PATH, "models")
NPZ_BASE_PATH = os.path.join(BASE_PATH, "Extracted_parameters")

assert os.path.exists(BASE_PATH), f"❌ Please create '{BASE_PATH}' folder in your Google Drive"
assert os.path.exists(MODEL_PATH), f"❌ Please create '{MODEL_PATH}' folder with SMPL-X model"
assert os.path.exists(NPZ_BASE_PATH), f"❌ Please create '{NPZ_BASE_PATH}' folder with NPZ files"

print("✓ Google Drive mounted successfully")
print(f"  Base path: {BASE_PATH}")
print(f"  Model path: {MODEL_PATH}")
print(f"  NPZ path: {NPZ_BASE_PATH}")

# List available videos
videos = [d for d in os.listdir(NPZ_BASE_PATH) if os.path.isdir(os.path.join(NPZ_BASE_PATH, d))]
print(f"\n✓ Found {len(videos)} videos: {videos}")

In [ ]:
# Cell 3: Load SMPL-X Model
import torch
import smplx
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Load SMPL-X model
print("\nLoading SMPL-X model...")
use_pca = False

try:
    model = smplx.create(
        model_path=MODEL_PATH,
        model_type='smplx',
        gender='neutral',
        num_betas=10,
        num_expression_coeffs=10,
        use_pca=False,
        use_face_contour=True,
        ext='npz'
    ).to(device)
    print("✓ Model loaded successfully (full hand articulation)")
except Exception as e:
    print(f"⚠ Falling back to PCA hands due to: {e}")
    model = smplx.create(
        model_path=MODEL_PATH,
        model_type='smplx',
        gender='neutral',
        num_betas=10,
        num_expression_coeffs=10,
        use_pca=True,
        num_pca_comps=12,
        use_face_contour=True,
        ext='npz'
    ).to(device)
    use_pca = True
    print("✓ Model loaded successfully (PCA hands)")

print(f"\nModel configuration:")
print(f"  Device: {device}")
print(f"  PCA hands: {use_pca}")
print(f"  Num vertices: {model.get_num_verts()}")
print(f"  Num faces: {len(model.faces)}")

In [ ]:
# Cell 4: Mesh Generation Functions
import trimesh
import pyrender
from PIL import Image

def load_params_safe(npz_path, person_id=0, use_pca=False):
    """Load SMPL-X parameters from NPZ file with error handling"""
    try:
        data = np.load(npz_path, allow_pickle=True)
        
        # Check if person exists
        person_ids = data.get('person_ids', np.array([]))
        if len(person_ids) == 0:
            return None, "no_person_detected"
        
        prefix = f'person_{person_id}_smplx_'
        
        # Check required keys
        required = ['root_pose', 'body_pose', 'shape', 'expr', 'jaw_pose']
        missing = [k for k in required if prefix + k not in data.files]
        if missing:
            return None, f"missing_keys:{','.join(missing)}"
        
        # Extract parameters
        params = {
            'global_orient': data[prefix + 'root_pose'].reshape(1, 3).astype(np.float32),
            'body_pose': data[prefix + 'body_pose'].reshape(1, -1).astype(np.float32),
            'jaw_pose': data[prefix + 'jaw_pose'].reshape(1, 3).astype(np.float32),
            'betas': data[prefix + 'shape'].reshape(1, -1).astype(np.float32),
            'expression': data[prefix + 'expr'].reshape(1, -1).astype(np.float32),
            'leye_pose': np.zeros((1, 3), dtype=np.float32),
            'reye_pose': np.zeros((1, 3), dtype=np.float32)
        }
        
        # Handle hand poses
        if use_pca:
            params['left_hand_pose'] = np.zeros((1, 12), dtype=np.float32)
            params['right_hand_pose'] = np.zeros((1, 12), dtype=np.float32)
        else:
            params['left_hand_pose'] = data[prefix + 'lhand_pose'].reshape(1, -1).astype(np.float32)
            params['right_hand_pose'] = data[prefix + 'rhand_pose'].reshape(1, -1).astype(np.float32)
        
        cam_trans = data.get(f'person_{person_id}_cam_trans')
        
        return (params, cam_trans), None
        
    except Exception as e:
        return None, f"error:{str(e)}"


def reconstruct_mesh(model, params, cam_trans, device):
    """Reconstruct 3D mesh from SMPL-X parameters"""
    try:
        with torch.no_grad():
            tensors = {k: torch.from_numpy(v).float().to(device) for k, v in params.items()}
            output = model(**tensors)
            vertices = output.vertices[0].cpu().numpy()
            faces = model.faces
        
        if cam_trans is not None:
            vertices += cam_trans
        
        return vertices, faces, None
    except Exception as e:
        return None, None, str(e)


def render_mesh(vertices, faces, img_size=720):
    """Render mesh to image"""
    try:
        os.environ['PYOPENGL_PLATFORM'] = 'egl'
        
        # Create trimesh
        mesh = trimesh.Trimesh(vertices=vertices, faces=faces)
        
        # Center and scale
        bounds = mesh.bounds
        center = bounds.mean(axis=0)
        mesh.vertices -= center
        scale = 2.0 / (bounds[1] - bounds[0]).max()
        mesh.vertices *= scale
        
        # Create pyrender mesh with material
        material = pyrender.MetallicRoughnessMaterial(
            baseColorFactor=[0.8, 0.8, 0.8, 1.0],
            metallicFactor=0.0,
            roughnessFactor=0.7
        )
        py_mesh = pyrender.Mesh.from_trimesh(mesh, material=material)
        
        # Create scene
        scene = pyrender.Scene(bg_color=[0.96, 0.96, 0.96, 1.0])
        scene.add(py_mesh)
        
        # Add camera
        camera = pyrender.PerspectiveCamera(yfov=np.pi / 3.0)
        camera_pose = np.array([
            [1.0, 0.0, 0.0, 0.0],
            [0.0, 1.0, 0.0, 0.0],
            [0.0, 0.0, 1.0, 2.5],
            [0.0, 0.0, 0.0, 1.0]
        ])
        scene.add(camera, pose=camera_pose)
        
        # Add lights
        light1 = pyrender.DirectionalLight(color=[1.0, 1.0, 1.0], intensity=3.0)
        light2 = pyrender.DirectionalLight(color=[1.0, 1.0, 1.0], intensity=2.0)
        
        scene.add(light1, pose=np.array([
            [1.0, 0.0, 0.0, 0.0],
            [0.0, 1.0, 0.0, 1.0],
            [0.0, 0.0, 1.0, 2.0],
            [0.0, 0.0, 0.0, 1.0]
        ]))
        
        scene.add(light2, pose=np.array([
            [1.0, 0.0, 0.0, -1.0],
            [0.0, 1.0, 0.0, -1.0],
            [0.0, 0.0, 1.0, 2.0],
            [0.0, 0.0, 0.0, 1.0]
        ]))
        
        # Render
        renderer = pyrender.OffscreenRenderer(img_size, img_size)
        color, _ = renderer.render(scene)
        renderer.delete()
        
        return color, None
    except Exception as e:
        return None, str(e)

print("✓ Mesh generation functions loaded")

In [ ]:
# Cell 5: Create FastAPI Server
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import base64
from io import BytesIO
import time

app = FastAPI(
    title="ECOLANG Mesh API",
    description="Real-time SMPL-X mesh generation from NPZ parameters",
    version="1.0.0"
)

# Request model
class RenderRequest(BaseModel):
    video_id: str
    frame_number: int

# Health check endpoint
@app.get("/health")
async def health_check():
    return {
        "status": "healthy",
        "device": str(device),
        "use_pca": use_pca,
        "model_loaded": model is not None,
        "timestamp": time.time()
    }

# List available videos
@app.get("/available_videos")
async def list_videos():
    try:
        videos = [d for d in os.listdir(NPZ_BASE_PATH) 
                 if os.path.isdir(os.path.join(NPZ_BASE_PATH, d))]
        return {"videos": videos, "count": len(videos)}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

# Main render endpoint
@app.post("/render_frame")
async def render_frame(request: RenderRequest):
    try:
        # Build NPZ path
        npz_path = os.path.join(
            NPZ_BASE_PATH,
            request.video_id,
            f"frame_{request.frame_number:04d}_params.npz"
        )
        
        # Check file exists
        if not os.path.exists(npz_path):
            raise HTTPException(
                status_code=404,
                detail=f"NPZ file not found: {npz_path}"
            )
        
        # Load parameters
        result, error = load_params_safe(npz_path, person_id=0, use_pca=use_pca)
        if error:
            raise HTTPException(
                status_code=400,
                detail=f"Failed to load parameters: {error}"
            )
        
        params, cam_trans = result
        
        # Reconstruct mesh
        vertices, faces, err = reconstruct_mesh(model, params, cam_trans, device)
        if err:
            raise HTTPException(
                status_code=500,
                detail=f"Mesh reconstruction failed: {err}"
            )
        
        # Render to image
        img, err = render_mesh(vertices, faces, img_size=720)
        if err:
            raise HTTPException(
                status_code=500,
                detail=f"Mesh rendering failed: {err}"
            )
        
        # Convert to base64
        buffered = BytesIO()
        Image.fromarray(img).save(buffered, format="PNG")
        img_base64 = base64.b64encode(buffered.getvalue()).decode()
        
        return {
            "success": True,
            "video_id": request.video_id,
            "frame_number": request.frame_number,
            "image": img_base64
        }
        
    except HTTPException:
        raise
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

print("✓ FastAPI server configured")

In [ ]:
# Cell 6: Start Server with ngrok Tunnel
import uvicorn
from pyngrok import ngrok
import nest_asyncio

nest_asyncio.apply()

# Start ngrok tunnel
print("Starting ngrok tunnel...")
public_url = ngrok.connect(8000)

print("\n" + "="*70)
print("🚀 ECOLANG Mesh API is LIVE!")
print("="*70)
print(f"\n📡 Public URL: {public_url}")
print(f"\n🔍 Test endpoints:")
print(f"   Health: {public_url}/health")
print(f"   Videos: {public_url}/available_videos")
print(f"   Docs: {public_url}/docs")
print("\n" + "="*70)
print("⚠️  IMPORTANT: Copy the URL above and add to Streamlit Cloud secrets:")
print(f"   COLAB_API_URL = \"{public_url}\"")
print("="*70)
print("\n💡 Keep this cell running to keep the API alive!")
print("   (Colab will timeout after ~12 hours of inactivity)\n")

# Start FastAPI server
uvicorn.run(app, host="0.0.0.0", port=8000, log_level="info")